In [ ]:
import os
import sys
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from datetime import datetime
from pathlib import Path
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib.patches import Rectangle

# for some conversion between thermodynamic variables
from metpy.units import units
from metpy.calc import thermo

import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
# data directoies
dir_era5 = Path('/neelin2020/ERA-5/NC_FILES/')
dir_buoy = Path('/neelin2020/ERA-5_buoy/layer_thetae/')

In [ ]:
def find_pbl_top_level(sp, temp, pbl_depth=100):
    
    if len(sp.squeeze().dims) == 0: # 1-dimension, grid data
    
        p_level = temp.level # get era-5 standard pressure levels
        idx = np.argmin(abs(level.values- (sp.values - pbl_depth)))
        
        pbl_top_level_xr = p_level[idx]
                
    elif len(sp.squeeze().dims) == 2: # 2-dimension (lat, lon)
        
        p_level_2d = (temp - temp + temp.level) 
        idx = np.argmin(abs(p_level_2d.values- (sp.values - pbl_depth)), axis=0)
        
        pbl_list = []
        for n in idx.ravel():
            pbl_list.append(p_level_2d.level[n])
        pbl_list = np.asarray(pbl_list)
        
        pbl_top_level = pbl_list.reshape((len(sp.latitude), len(sp.longitude)))
        #convert back into xarray
        pbl_top_level_xr = xr.Dataset(data_vars=dict(pbl_top_level=(['latitude','longitude'],pbl_top_level)),
                                      coords=dict(latitude=(['latitude'], sp.latitude.values),
                                                  longitude=(['longitude'], sp.longitude.values)))
            
    return pbl_top_level_xr

In [ ]:
def es_calc_bolton(temp):
    # in hPa

    tmelt  = 273.15
    tempc = temp - tmelt 
    es = 6.112*np.exp(17.67*tempc/(243.5+tempc))
    
    return es

In [ ]:
def es_calc(temp):

    tmelt  = 273.15
    
    c0=0.6105851e+03
    c1=0.4440316e+02
    c2=0.1430341e+01
    c3=0.2641412e-01
    c4=0.2995057e-03
    c5=0.2031998e-05
    c6=0.6936113e-08
    c7=0.2564861e-11
    c8=-.3704404e-13

    tempc = temp - tmelt 
    tempcorig = tempc
        
    #if tempc < -80:
    es_ltn80c = es_calc_bolton(temp)
    es_ltn80c = es_ltn80c.where(tempc < -80, 0)
    #else:
    es = c0+tempc*(c1+tempc*(c2+tempc*(c3+tempc*(c4+tempc*(c5+tempc*(c6+tempc*(c7+tempc*c8)))))))
    es_gtn80c = es/100
    es_gtn80c = es_gtn80c.where(tempc >= -80, 0)
    
    # complete es
    es = es_ltn80c + es_gtn80c
    
    return es

In [ ]:
def qs_calc(temp):

    tmelt  = 273.15
    RV=461.5
    RD=287.04

    EPS=RD/RV

    press = temp.level * 100. # in Pa
    tempc = temp - tmelt 

    es=es_calc(temp) 
    es=es * 100. #hPa
    qs = (EPS * es) / (press + ((EPS-1.)*es))
    
    return qs

In [ ]:
def theta_e_calc(temp, q):
    
    # following the definitions in Bolton (1980): the calculation of equivalent potential temperature
    
    pref = 100000.
    tmelt  = 273.15
    CPD=1005.7
    CPV=1870.0
    CPVMCL=2320.0
    RV=461.5
    RD=287.04
    EPS=RD/RV
    ALV0=2.501E6

    press = temp.level * 100. # in Pa
    tempc = temp - tmelt # in C

    r = q / (1. - q) 

    # get ev in hPa 
    ev_hPa = temp.level * r / (EPS + r) # hpa

    #get TL
    TL = (2840. / ((3.5*np.log(temp)) - (np.log(ev_hPa)) - 4.805)) + 55.

    #calc chi_e:
    chi_e = 0.2854 * (1. - (0.28 * r))

    theta_e = temp * np.power((pref / press),chi_e) * np.exp(((3.376/TL) - 0.00254) * r * 1000 * (1. + (0.81 * r)))
    
    return theta_e

In [ ]:
def BL_estimates_cal_new(T, q, sp, T2m, q2m):
    """
    function for calcultinig the low-trospospheric buoyancy estimates
    T, q : 3D xarray dataarray (level, lat, lon)
    sp: surface pressure (lat, lon), unit: hPa
    T2m, q2m: temperature, specific humidity at 2m (lat, lon) 
    """
    
    T = T.drop('time')
    q = q.drop('time')
    T2m = T2m.drop('time')
    q2m = q2m.drop('time')
    sp = sp.drop('time')
    
    # constants
    Lv = 2.5e6 # (J/kg)
    g = 9.81 # (kg/m^2)
    cpd = 1004 # (J/kg/K)
    p0 = 1000  # (hPa)
    Rd = 287.15 # (J/kg)

    # find pbl top (100 hPa above the surface)
    pbl_top_level = find_pbl_top_level(sp, T, pbl_depth=100)
    
    thetae_bl_list = []
    thetae_lt_list = []
    thetae_sat_lt_list = []
    
    # loop for lat-lon grids
    for idx_lat in range(len(q2m.latitude)):
        for idx_lon in range(len(q2m.longitude)):
                        
            try:
                sf_p = sp.isel(latitude=idx_lat, longitude=idx_lon) # surface pressure 
                pbl_p = pbl_top_level.isel(latitude=idx_lat, longitude=idx_lon).pbl_top_level.values # pbl top pressure
                T_sf = T2m.isel(latitude=idx_lat, longitude=idx_lon).values
                q_sf = q2m.isel(latitude=idx_lat, longitude=idx_lon).values

                T_at_sf = T2m.isel(latitude=idx_lat, longitude=idx_lon).values
                q_at_sf = q2m.isel(latitude=idx_lat, longitude=idx_lon).values
                T_above_sf = T.isel(latitude=idx_lat, longitude=idx_lon).sel(level=slice(100,int(sf_p))).values
                q_above_sf = q.isel(latitude=idx_lat, longitude=idx_lon).sel(level=slice(100,int(sf_p))).values

                # reconstruct T, q profile by adding surface quantities
                T_1d = np.concatenate([np.array([T_at_sf]), np.flip(T_above_sf)])            
                q_1d = np.concatenate([np.array([q_at_sf]), np.flip(q_above_sf)])
                pressure_1d = np.concatenate([np.array([sf_p]), np.flip(T.sel(level=slice(100,int(sf_p))).level.values)])
                T_1d_xr = xr.DataArray(data=T_1d,
                                       dims=["level"],
                                       coords=dict(level=(["level"], pressure_1d)))
                q_1d_xr = xr.DataArray(data=q_1d,
                                       dims=["level"],
                                       coords=dict(level=(["level"], pressure_1d)))

                # splitting into boundary layer and lower free troposphere
                # 1. boundary layer, bl
                q_bl = q_1d_xr.sel(level=slice(int(sf_p), pbl_p))
                T_bl = T_1d_xr.sel(level=slice(int(sf_p), pbl_p))
                # 2. lower free troposphere, lt
                q_lt = q_1d_xr.sel(level=slice(pbl_p,pbl_p-400))
                T_lt = T_1d_xr.sel(level=slice(pbl_p,pbl_p-400))     

                # calculating layer-averaged thetae components
                thetae_bl = -theta_e_calc(T_bl, q_bl).integrate('level')/(T_bl.level[0]-T_bl.level[-1]) # negative sign b.c. decreasing p
                thetae_lt = -theta_e_calc(T_lt, q_lt).integrate('level')/(T_lt.level[0]-T_lt.level[-1])
                qsat_lt = qs_calc(T_lt)
                thetae_sat_lt = -theta_e_calc(T_lt, qsat_lt).integrate('level')/(T_lt.level[0]-T_lt.level[-1]) 

                thetae_bl_list.append(thetae_bl.values)
                thetae_lt_list.append(thetae_lt.values)
                thetae_sat_lt_list.append(thetae_sat_lt.values)
                
            except:
                
                thetae_bl_list.append(np.nan)
                thetae_lt_list.append(np.nan)
                thetae_sat_lt_list.append(np.nan)
            
    # convert to geolocated xarray
    thetae_bl_xr = xr.DataArray(data = np.asarray(thetae_bl_list).reshape((len(T.latitude), len(T.longitude))),
                               dims = ["latitude",'longitude'],
                               coords = dict(latitude=(["latitude"], T.latitude.values),
                                           longitude=(["longitude"], T.longitude.values)))
    thetae_lt_xr = xr.DataArray(data = np.asarray(thetae_lt_list).reshape((len(T.latitude), len(T.longitude))),
                               dims = ["latitude",'longitude'],
                               coords = dict(latitude=(["latitude"], T.latitude.values),
                                           longitude=(["longitude"], T.longitude.values)))
    thetae_sat_lt_xr = xr.DataArray(data = np.asarray(thetae_sat_lt_list).reshape((len(T.latitude), len(T.longitude))),
                               dims = ["latitude",'longitude'],
                               coords = dict(latitude=(["latitude"], T.latitude.values),
                                           longitude=(["longitude"], T.longitude.values)))
    # calculate buoyancy estimates
    delta_pl=400
    delta_pb=100
    wb=(delta_pb/delta_pl)*np.log((delta_pl+delta_pb)/delta_pb)
    wl=1-wb
    
    # calculate buoyancy estimate
    Buoy_CAPE = ((thetae_bl_xr-thetae_sat_lt_xr)/thetae_sat_lt_xr)*340
    Buoy_CAPE_xr = Buoy_CAPE.rename('Buoy_CAPE').to_dataset()
    Buoy_SUBSAT = ((thetae_sat_lt_xr-thetae_lt_xr)/thetae_sat_lt_xr)*340
    Buoy_SUBSAT_xr = Buoy_SUBSAT.rename('Buoy_SUBSAT').to_dataset()
    Buoy_TOT = (9.81/(340*3))*((wb*Buoy_CAPE)-(wl*Buoy_SUBSAT))
    Buoy_TOT_xr = Buoy_TOT.rename('Buoy_TOT').to_dataset()
    
    thetae_bl_xr  = thetae_bl_xr.rename('thetae_bl').to_dataset()
    thetae_lt_xr  = thetae_lt_xr.rename('thetae_lt').to_dataset()
    thetae_sat_lt_xr  = thetae_sat_lt_xr.rename('thetae_sat_lt').to_dataset()
    
    return xr.merge([Buoy_CAPE_xr, Buoy_SUBSAT_xr, Buoy_TOT_xr, thetae_bl_xr, thetae_lt_xr, thetae_sat_lt_xr])

In [ ]:
%%time
# calculate based on t2m, td2m, sp for boundary 
data_T = xr.open_dataset(dir_era5 / '2014/era-5.T.2014.01.nc')
data_T = data_T.reindex(latitude=list(reversed(data_T.latitude)))
data_T = data_T.t.sel(latitude=slice(-23,-13), longitude=slice(28.5,38.5))

data_q = xr.open_dataset(dir_era5 / '2014/era-5.q.2014.01.nc')
data_q = data_q.reindex(latitude=list(reversed(data_q.latitude)))
data_q = data_q.q.sel(latitude=slice(-23,-13), longitude=slice(28.5,38.5))

data_sp = xr.open_dataset(dir_era5/ '2014/era-5.sp.2014.01.nc')
data_sp = data_sp.reindex(latitude=list(reversed(data_sp.latitude)))
data_sp = data_sp.sp.sel(latitude=slice(-23,-13), longitude=slice(28.5,38.5))/100

data_t2m = xr.open_dataset(dir_era5/ '2014/era-5.2t.2014.01.nc')
data_t2m = data_t2m.reindex(latitude=list(reversed(data_t2m.latitude)))
data_t2m = data_t2m.VAR_2T.sel(latitude=slice(-23,-13), longitude=slice(28.5,38.5))

data_d2m = xr.open_dataset(dir_era5/ '2014/era-5.2d.2014.01.nc')
data_d2m = data_d2m.reindex(latitude=list(reversed(data_d2m.latitude)))
data_d2m = data_d2m.VAR_2D.sel(latitude=slice(-23,-13), longitude=slice(28.5,38.5))

#data_sp = data_surface.sp/100
#data_t2m = data_surface.t2m
#data_d2m = data_surface.d2m
data_q2m = thermo.specific_humidity_from_dewpoint(data_sp*100 * units.pascal, data_d2m * units.kelvin) 

idt = 23
# # derive low-tropospheric buoyancy estimates
BL_merged = BL_estimates_cal_new(data_T.isel(time=idt), data_q.isel(time=idt), data_sp.isel(time=idt)
                                 , data_t2m.isel(time=idt), data_q2m.isel(time=idt))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,5),subplot_kw={'projection': ccrs.PlateCarree()})
    
ax.coastlines(color='k',linewidth=0.7)
ax.add_feature(cfeat.LAND,zorder=3,edgecolor='grey',facecolor='none',linewidth=0.3) # maskout LAND
ax.add_feature(cfeat.BORDERS,zorder=3,edgecolor='grey',facecolor='none',linewidth=0.3)
ax.add_feature(cfeat.STATES,zorder=3,edgecolor='grey',facecolor='none',linewidth=0.3) 

ax.pcolormesh(BL_merged.longitude, BL_merged.latitude, BL_merged.Buoy_TOT, vmin=-0.12,vmax=0.02, cmap='jet',
              transform=ccrs.PlateCarree())

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='k', alpha=0.4, linestyle=':')
gl.xlabels_top = False
gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 8, 'color': 'k'}
gl.ylabel_style = {'size': 8, 'color': 'k'}

In [ ]:
BL_merged.Buoy_TOT.plot(vmin=-0.09, vmax=-0.02, cmap='jet')

In [ ]:
delta_pl=400
delta_pb=100
wb=(delta_pb/delta_pl)*np.log((delta_pl+delta_pb)/delta_pb)
wl=1-wb

BUOY_data = xr.open_dataset('/neelin2020/ERA-5_buoy/layer_thetae/era5_2layers_thetae_2014_01_01.nc')
BUOY_sub = BUOY_data.sel(time=datetime(2014,1,1,0),lon=slice(30,40), lat=slice(-20,-10))
thetae_bl = BUOY_sub.thetae_bl
thetae_sat_lt = BUOY_sub.thetae_sat_lt
thetae_lt = BUOY_sub.thetae_lt
BL_ori=(9.81/3)*(wb*(thetae_bl-thetae_sat_lt)/thetae_sat_lt-wl*(thetae_sat_lt-thetae_lt)/thetae_sat_lt)

In [ ]:
BL_ori.plot(vmin=-0.09, vmax=-0.02, cmap='jet')

In [ ]:
year = 2014
track_number = 84
dir_env_track = Path('/neelin2020/mcs_flextrkr/mcs_stats/envs_track/{}/tropics'.format(year))
tmp = xr.open_dataset(dir_env_track / 'mcs_era5_3D_envs_{}.{}.LD.nc'.format(year, str(track_number).zfill(5)))

In [ ]:
tmp.sel(mcs_phase='Init').thetae_lt.plot(vmin=332,vmax=345)

In [ ]:
pbl_top_level = find_pbl_top_level(data_sp.isel(time=idt), data_T.isel(time=idt), pbl_depth=100)

In [ ]:
sp = data_sp.isel(time=idt)
T2m = data_t2m.isel(time=idt)
q2m = data_q2m.isel(time=idt)
T = data_T.isel(time=idt)
q = data_q.isel(time=idt)

idx_lat = 0
idx_lon = 0

sf_p = sp.isel(latitude=idx_lat, longitude=idx_lon) # surface pressure 
pbl_p = pbl_top_level.isel(latitude=idx_lat, longitude=idx_lon).pbl_top_level.values # pbl top pressure
T_sf = T2m.isel(latitude=idx_lat, longitude=idx_lon).values
q_sf = q2m.isel(latitude=idx_lat, longitude=idx_lon).values

T_at_sf = T2m.isel(latitude=idx_lat, longitude=idx_lon).values
q_at_sf = q2m.isel(latitude=idx_lat, longitude=idx_lon).values
T_above_sf = T.isel(latitude=idx_lat, longitude=idx_lon).sel(level=slice(100,int(sf_p))).values
q_above_sf = q.isel(latitude=idx_lat, longitude=idx_lon).sel(level=slice(100,int(sf_p))).values

# reconstruct T, q profile by adding surface quantities
T_1d = np.concatenate([np.array([T_at_sf]), np.flip(T_above_sf)])            
q_1d = np.concatenate([np.array([q_at_sf]), np.flip(q_above_sf)])
pressure_1d = np.concatenate([np.array([sf_p]), np.flip(T.sel(level=slice(100,int(sf_p))).level.values)])
T_1d_xr = xr.DataArray(data=T_1d,
                       dims=["level"],
                       coords=dict(level=(["level"], pressure_1d)))
q_1d_xr = xr.DataArray(data=q_1d,
                       dims=["level"],
                       coords=dict(level=(["level"], pressure_1d)))

# splitting into boundary layer and lower free troposphere
# 1. boundary layer, bl
q_bl = q_1d_xr.sel(level=slice(int(sf_p), pbl_p))
T_bl = T_1d_xr.sel(level=slice(int(sf_p), pbl_p))
# 2. lower free troposphere, lt
q_lt = q_1d_xr.sel(level=slice(pbl_p,pbl_p-400))
T_lt = T_1d_xr.sel(level=slice(pbl_p,pbl_p-400))     

# calculating layer-averaged thetae components
thetae_bl = -theta_e_calc(T_bl, q_bl).integrate('level')/(T_bl.level[0]-T_bl.level[-1]) # negative sign b.c. decreasing p
thetae_lt = -theta_e_calc(T_lt, q_lt).integrate('level')/(T_lt.level[0]-T_lt.level[-1])
qsat_lt = qs_calc(T_lt)
thetae_sat_lt = -theta_e_calc(T_lt, qsat_lt).integrate('level')/(T_lt.level[0]-T_lt.level[-1]) 

In [ ]:
T_above_sf = T.isel(latitude=idx_lat, longitude=idx_lon).sel(level=slice(100, int(sf_p))).values
T_above_sf

In [ ]:
T

In [ ]:
pressure_1d